In [359]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [360]:
import cudf as cd
import cupy as cp

In [361]:
train_data = cd.read_csv('../input/new-dataset/train_1.csv')
test_data = cd.read_csv('../input/new-dataset/test_1.csv')

In [362]:
train_data.head()

In [363]:
train_data.shape

In [364]:
train_data = train_data.dropna(axis = 1)
train_data.head()

In [365]:
train_info = train_data.info()
train_info

In [366]:
y = train_data['SalePrice']
x = train_data.iloc[:, :-1]
x.head()

In [367]:
x.head()

In [368]:
y.head()

In [369]:
str_cols = cd.DataFrame()
for i in x.columns:
    if x[i].dtype == object:
        str_cols[i] = x[i]
        x.drop(i, 1, inplace = True)
    

In [370]:
x1 = x

In [371]:
print(x1.shape)
print(y.shape)

In [372]:
y[y[:] == y.isna()]

In [373]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
x1 = scaler.fit_transform(x1)

In [374]:
from cuml.preprocessing import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x1, y, test_size = 0.3, random_state = 42)

In [375]:
algos = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [376]:
from cuml import LinearRegression
import cuml
err = 9999999999
final_lr = None
for i in algos:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(x_train,y_train)
    preds = lr.predict(x_test)
    y_test=y_test.astype(cp.float64)
#     print(preds.isna() == True)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test,preds))
    if err>=cuml.metrics.regression.mean_squared_error(y_test,preds):
        err = cuml.metrics.regression.mean_squared_error(y_test,preds)
        final_lr = lr
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test,preds))

In [377]:
output = cd.DataFrame()
output["Id"] = test_data['Id']

In [378]:
test_data = test_data[x.columns]
test_data.fillna(0.0)

In [379]:
test_data['BsmtFinSF1'].fillna(0, inplace = True)
test_data['BsmtFinSF2'].fillna(0, inplace = True)
test_data[test_data['BsmtFinSF1'].isnull() == True]


In [380]:
output["SalePrice"] = pred

In [381]:
output.to_csv('submission.csv', index=False)